In [4]:
# %pip install Pillow
# %pip install opencv-python

import os
from PIL import Image

from lib.cutter import find_biggest_white
from lib.cleaner import clean_noise

In [ ]:
PICTURE_FOLDER = "./pictures/"          # slozka s obrazky
RESULT_FILE_NAME = "result"
RESULT_PATH = f"./vysledek/{RESULT_FILE_NAME}.png"
RESULT_PATH_MULTI = f"./vysledek/{RESULT_FILE_NAME}%s.png"

folder_with_pictures = PICTURE_FOLDER
pictures: list[Image] = []
dimensions: list[tuple[int, int]] = []

print(f"Zpracovávám obrázky z '{folder_with_pictures}'")
print(sorted(os.listdir(folder_with_pictures)))

for file in sorted(os.listdir(folder_with_pictures)):
    if file.lower().endswith((".png", ".jpg", ".jpeg")):
        file_path = os.path.join(folder_with_pictures, file)
        print(f"Otevírám soubor '{file_path}'")
        try:
            picture = find_biggest_white(file_path)
            aspect_ratio = picture.width / picture.height
            target_height = int(1920 / aspect_ratio)
            # Přeskalování obrázku
            resized_img = picture.resize((1920, target_height))

            # picture = clean_noise(picture)
            dimensions.append(resized_img.size)
        except Exception as e:
            print("Vyskytla se chyba: ", e)
            raise e

        pictures.append(resized_img)

limit = 2950
pictures_per_photo = [0]

height = 0
for i in range(len(pictures)):
    height += pictures[i].height
    if height > limit:
        pictures_per_photo.append(i)
        height = pictures[i].height

if len(pictures_per_photo) == 1:
    # Vytvoření prázdného obrázku
    heights = [picture.height for picture in pictures]
    result_picture = Image.new("RGB", (1920, sum(heights)))

    # Kopírování každého obrázku pod sebe
    height_breakpoint = 0
    for i, picture in enumerate(pictures):
        result_picture.paste(picture, (0, height_breakpoint))
        height_breakpoint += heights[i]

    # Uložení výsledného obrázku
    result_picture.save(RESULT_PATH)
    print(f"  - uloženo {RESULT_PATH}")

else:
    for current, next_breakpoint in zip(pictures_per_photo, pictures_per_photo[1:]):
        current_pictures = pictures[current:next_breakpoint]

        heights = [picture.height for picture in current_pictures]
        result_picture = Image.new("RGB", (1920, sum(heights)))

        # Kopírování každého obrázku pod sebe
        height_breakpoint = 0
        for i, picture in enumerate(current_pictures):
            result_picture.paste(picture, (0, height_breakpoint))
            height_breakpoint += heights[i]

        # Uložení výsledného obrázku
        result_picture.save(RESULT_PATH_MULTI % next_breakpoint)
        print(f"  - uloženo {RESULT_PATH_MULTI % next_breakpoint}")

    # od posledniho indexu do konce
    current_pictures = pictures[pictures_per_photo[-1]:]
    heights = [picture.height for picture in current_pictures]
    result_picture = Image.new("RGB", (1920, sum(heights)))

    # Kopírování každého obrázku pod sebe
    height_breakpoint = 0
    for i, picture in enumerate(current_pictures):
        result_picture.paste(picture, (0, height_breakpoint))
        height_breakpoint += heights[i]

    # Uložení výsledného obrázku
    result_picture.save(RESULT_PATH_MULTI % len(pictures))
    print(f"  - uloženo {RESULT_PATH_MULTI % len(pictures)}")
